In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns
from random import sample, seed 

from scipy import stats

import matplotlib.pyplot as plt


# Read the data

In [26]:
#Read omics data
omics_gen_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/EUROS_PRS_9_disease_send.csv'
PCA_dataset_path = '/data/sharedData/Euros_genetic_principal_components.csv'

omics = pd.read_csv(omics_gen_path, sep = ';', index_col = 0)
PCA_dataset = pd.read_csv(PCA_dataset_path, sep = ';', index_col = 0)



boxplot_data_full = ({'prob':[],
                'reference':[],
                'disease':[],
                'data_type':[],
                'Model': []
                })

boxplot_data_full = pd.DataFrame(boxplot_data_full)

Shiny_data = ({'prob':[],
                'reference':[],
               'Disease':[],
               'Data': [],
               'AUC_train':[],
               'AUC_test':[],
               'Model':[]
              })
Shiny_data = pd.DataFrame(Shiny_data)


for disease in ['CD', 'UC', 'RA', 'PSO', 'SLE', 'COPD', 'obesity', 'T2D', 'Atherosclerosis']:
#for disease in ['CD', 'UC']:

    data_type = 'gen'
        
    if disease == 'Atherosclerosis':
        colname_disease = 'COPD'
    else:
        colname_disease = disease
        
    PRS = [col for col in omics.columns if colname_disease in col]
    omics_specificPRS = omics[PRS]
        
        
    for model_type in ['incident', 'prevalent']:           


        if model_type == 'incident':
            sick_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_all_incident.csv'    
            clinical_sick = pd.read_csv(sick_path, index_col = 0)
            
            HC_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_all_HC_PairedToIncident.csv'
            clinical_healthy = pd.read_csv(HC_path, index_col = 0)
        if model_type == 'prevalent':
            sick_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_all_prevalent.csv'    
            clinical_sick = pd.read_csv(sick_path, index_col = 0)
            
            HC_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_all_HC_PairedToPrevalent.csv'
            clinical_healthy = pd.read_csv(HC_path, index_col = 0)


        # clinical -> clinical data from both healthy and sick
        # omics_clinical -> subset of genomics including only patients from clinical data
        clinical = pd.concat([clinical_sick,clinical_healthy])

        omics_clinical = clinical.join(PCA_dataset).join(omics_specificPRS)
        omics_clinical['PC3'] = pd.to_numeric(omics_clinical['PC3'].str.replace(',', '').str.replace('E', 'e'), errors='coerce')

                #devide between healthy and sick
        X_train, X_test, y_train, y_test = train_test_split(omics_clinical.drop(columns=['group']),
                                                            omics_clinical['group'],
                                                            test_size=0.3,
                                                            random_state=42)

        X_train = pd.get_dummies(X_train)
        X_test = pd.get_dummies(X_test)


        clf_ridge = LogisticRegressionCV(cv=10, Cs = 10, random_state=42, max_iter = 10000,
                                    penalty='l2').fit(X_train, y_train)



        #compute feature importance 
        coefficients = clf_ridge.coef_[0]
        feature_importance = pd.DataFrame({'Feature': X_train.columns,
                                            'Importance': np.abs(coefficients),
                                            'Coef': coefficients})
        feature_importance = feature_importance.sort_values('Importance', ascending = True)


        #compute metrics
        auc_train = roc_auc_score(y_train, clf_ridge.predict_proba(X_train)[:, 1])
        auc = roc_auc_score(y_test, clf_ridge.predict_proba(X_test)[:, 1])
        conf_matrix = confusion_matrix(y_test, clf_ridge.predict(X_test), labels = np.unique(y_test))


        #print summary
        print('Disease: ' + disease)
        print(conf_matrix)
        print('AUC train: ' + str(auc_train))
        print('AUC: ' + str(auc))
        print('\n')



             
        # Prediction for sick/healthy
        X_test_summary = pd.DataFrame(clf_ridge.predict_proba(X_test)[:, 1], columns = ['prob'])
        X_test_summary['reference'] = y_test.values
                
        boxplot_data = X_test_summary[['prob', 'reference']]
        boxplot_data['disease'] = disease
        boxplot_data['data_type'] = data_type
        boxplot_data['Model'] = model_type
                
        boxplot_data_full = boxplot_data_full.append(boxplot_data, ignore_index=True)

        new_row = {'prob': list(np.round(boxplot_data['prob'].astype(float), 3)),
                    'reference': list(boxplot_data['reference']),
                    'Disease': disease,
                    'Data': data_type,
                    'AUC_train': auc_train,
                    'AUC_test': auc,
                    'Model': model_type
            }

        Shiny_data = Shiny_data.append(new_row, ignore_index=True)
        
Shiny_data = Shiny_data[['Disease', 'Data', 'Model', 'AUC_train', 'AUC_test', 'prob', 'reference']]


/home/marsm05/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Disease: CD
[[258 165]
 [167 251]]
AUC train: 0.6226775427669149
AUC: 0.6245489610551201


Disease: CD
[[227 104]
 [133 198]]
AUC train: 0.6826329274057477
AUC: 0.7047033159609715


Disease: UC
[[403 336]
 [362 407]]
AUC train: 0.584067584270244
AUC: 0.5517525352328296


Disease: UC
[[351 229]
 [292 341]]
AUC train: 0.6035839064690507
AUC: 0.6014326959742877


Disease: RA
[[1146  741]
 [1077  861]]
AUC train: 0.5289935879284972
AUC: 0.533115340800644


Disease: RA
[[387 227]
 [325 305]]
AUC train: 0.5592955029308859
AUC: 0.5579494338451993


Disease: PSO
[[777 400]
 [636 570]]
AUC train: 0.6082527658726203
AUC: 0.5925195602277482


Disease: PSO
[[202  97]
 [120 172]]
AUC train: 0.6678193850205512
AUC: 0.6740504879277959


Disease: SLE
[[60 39]
 [71 52]]
AUC train: 0.5902288467329119
AUC: 0.5434836166543484


Disease: SLE
[[33 24]
 [34 23]]
AUC train: 0.6094182825484764
AUC: 0.5130809479839951


Disease: COPD
[[2495 2110]
 [1819 2885]]
AUC train: 0.601488248932729
AUC: 0.603309355772710

In [38]:
Shiny_data.to_csv('ShinyData_clinical_story_genomics.csv') # Supplementary data 2 (genomics)

In [40]:
boxplot_data_full.to_csv('graph_data_clinical_story_genomics.csv')